Useful links:
 - https://stackoverflow.com/questions/19846332/python-threading-inside-a-class/19846691
 

In [1]:
# Research while working on 1020 project
import socket
import threading
import time
import pickle

In [2]:
# Constants
HEADERSIZE = 10

In [3]:
class server():
    def __init__(self, ip, port):
        self.ip = ip
        self.port = port
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.socket.bind((self.ip, self.port))
        self.socket.listen(5)
        
        self.accepting_thread = None
        self.clients = []
        self.client_threads = []
        
        self.stop_threads = False
    
    def broadcast(self, message):
        pass
    
    def _accept_clients(self):
        while not self.stop_threads:
            sockfd, addr = self.socket.accept()
            self.clients.append(sockfd)
            self.client_threads.append(threading.Thread(target=self._client_thread, args=(sockfd,), daemon=True))
            self.client_threads[-1].start()
            print(self.clients)
        
    def accept_clients(self):
        self.accepting_thread = threading.Thread(target=self._accept_clients, daemon=True)
        self.accepting_thread.start()
        
    def _client_thread(self, socketfd):
        while not self.stop_threads:
            data = bytes("", 'utf-8')
            try:
                incoming_size = int(socketfd.recv(HEADERSIZE))
                #data = socketfd.recv(incoming_size)
                while len(data) < incoming_size:
                    data += socketfd.recv(16)
                if len(data) == incoming_size:
                    print(str(data))
            except Exception as e:
                print(e)
                socketfd.close()
    
    def stop(self):
        self.stop_threads = True
                
    
                
        

class client():
    def __init__(self, username, server_address):
        self.username = username
        self.server_address = server_address
        self.socket = socket.socket()

In [4]:
def create_message(obj):
    compressed_text = pickle.dumps(obj)
    #compressed_text = bytes(text, 'utf-8')
    length = len(compressed_text)
    return bytes(f"{length:<{HEADERSIZE}}", 'utf-8') + compressed_text

In [5]:
# Testing
test_server = server('127.0.0.1', 1242)
test_server.accept_clients()
#print(create_message([1,2,3]))


[<socket.socket fd=43, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 1242), raddr=('127.0.0.1', 58742)>]b'f'

b'f'
b'For i in range(100)'
b'IT WORKS!!!!!! WITH AS MUCH TEXT AS I WANT. AT LEAST UP TO 1024 CHARS'
b'What if I send more?'
b'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa